In [230]:
import pandas as pd
import statsapi
from pprint import pprint
import requests
from bs4 import BeautifulSoup

In [245]:
espn_url = "https://www.espn.com/mlb/war/leaders/_/type/seasonal/year/20"

In [261]:
year_list = ['19', '18', '17', '16', '15']
df_list = []

In [262]:
for year in year_list:
    query_url = espn_url+year
    qdfl = pd.read_html(query_url)
    qdfl[0]['season'] = year
    df_list.append(qdfl[0])

In [263]:
data = pd.concat(df_list)
data.head()

,0,1,2,3,4,5,6,7,8,9,season
0,2019 Season WAR Leaders,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
1,RK,PLAYER,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,19
2,1,Cody Bellinger,9,6.6,1.9,7,5,4.7,72.9,0.5,19
3,2,Alex Bregman,8.4,7.7,1.2,6.1,5.3,5,62,0.5,19
4,3,Mike Trout,8.3,8.3,0.3,6.3,5.5,5,65.7,0.5,19


In [264]:
cols = ['rank', 'player', 'WAR', 'OFF', 'DEF', 'WAA', 'TRPG', 'ORPG', 'RAA', 'WAAWP', 'Season']

In [265]:
data.columns = cols
del data['rank']
data.head()

,player,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,Season
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
1,PLAYER,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,19
2,Cody Bellinger,9,6.6,1.9,7,5,4.7,72.9,0.5,19
3,Alex Bregman,8.4,7.7,1.2,6.1,5.3,5,62,0.5,19
4,Mike Trout,8.3,8.3,0.3,6.3,5.5,5,65.7,0.5,19


In [266]:
data = data.dropna(subset=['player'])

In [267]:
data = data[~data.player.str.contains("WAR")]

In [268]:
data.head()

,player,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,Season
1,PLAYER,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,19
2,Cody Bellinger,9,6.6,1.9,7,5,4.7,72.9,0.5,19
3,Alex Bregman,8.4,7.7,1.2,6.1,5.3,5,62,0.5,19
4,Mike Trout,8.3,8.3,0.3,6.3,5.5,5,65.7,0.5,19
5,Marcus Semien,8.1,7.5,1.5,5.8,5.3,5,58.5,0.5,19


In [269]:
# new data frame with split value columns 
new = data["player"].str.split(" ", n = 1, expand = True) 
  
# making separate first name column from new data frame 
data["First Name"]= new[0] 
data["Last Name"]= new[1] 
  
data = data.drop(data.index[0])
data.reset_index()
data.head()

,player,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,Season,First Name,Last Name
2,Cody Bellinger,9,6.6,1.9,7,5,4.7,72.9,0.5,19,Cody,Bellinger
3,Alex Bregman,8.4,7.7,1.2,6.1,5.3,5,62,0.5,19,Alex,Bregman
4,Mike Trout,8.3,8.3,0.3,6.3,5.5,5,65.7,0.5,19,Mike,Trout
5,Marcus Semien,8.1,7.5,1.5,5.8,5.3,5,58.5,0.5,19,Marcus,Semien
6,Justin Verlander,7.8,0,0,5.8,5,3.4,56.6,0.7,19,Justin,Verlander


In [271]:
data.drop_duplicates(subset ="player", keep ='first', inplace = True)

In [272]:
data.to_csv('war_leaders.csv')

In [276]:
id_list = pd.read_csv('master_id_list.csv', encoding='latin-1')
id_df = pd.DataFrame(id_list)[['mlb_name','espn_id','espn_name', 'espn_pos']].dropna()


In [278]:
new_data = pd.merge(data, id_df,  how='left', left_on=['player'], right_on = ['mlb_name'])

In [280]:
#do not overwrite this csv!!!
#new_data.to_csv('player_list.csv')

In [13]:
clean_data = pd.read_csv('player_list.csv')

In [14]:
q_urls=[]

In [15]:
for index, row in clean_data.iterrows():
    espn_tag = row['First Name']+ '-' +row['Last Name']
    espn_tag = espn_tag.lower()
    turl = 'http://www.espn.com/mlb/player/stats/_/id/' + str(row['espn_id']) + '/'+ espn_tag
    q_urls.append(turl)

In [16]:
clean_data['qurl'] = q_urls

In [17]:
mongo_list = []
for i in clean_data.index:
    temp_df = pd.read_html(clean_data['qurl'][i])
    #above returns a list of dfs, this line pulls the df that contains the stat table
    stat_df = temp_df[1]
    #dropping the blank row at the top and the summary row at the bottom
    stat_df.drop(stat_df.tail(1).index,inplace=True)
    stat_df.drop(stat_df.head(1).index,inplace=True)
    #turning the df into a dictionary and adding to the mongo listS
    mongo_list.append(stat_df.to_dict('series'))

In [18]:
for i in range(len(clean_data)):
    mongo_list[i]['name'] = clean_data['player'][i]

In [19]:
%store mongo_list
    



Stored 'mongo_list' (list)
